In [1]:
from pyspark.sql import SparkSession
from pyspark import SparkConf, __version__ as pyspark_version

In [2]:
main_pyspark_version = ".".join(pyspark_version.split(".")[:-1])
KAFKA_HOST = "172.19.0.3:9094"
#packages = f"org.apache.spark:spark-sql-kafka-0-10_2.12:{main_pyspark_version}.0"
packages = f"org.apache.spark:spark-sql-kafka-0-10_2.12:3.2.4"
topic_name = "test"

In [3]:
if __name__ == "__main__":
    spark = (
        SparkSession.builder.appName("Kafka Pyspark Streaming Learning")
        .master("local[*]")
        .config("spark.jars.packages",packages)
        .config("spark.sql.warehouse.dir","file:///spark-warehouse")
        .getOrCreate()
    )
    spark.sparkContext.setLogLevel("ERROR")

In [4]:
kafkaParams = {
    "kafka.bootstrap.servers": "172.19.0.3:9094",  # Replace with your broker addresses
    "subscribe":f"{topic_name}",  # Replace with your topic name
    "startingOffsets": "earliest",  # or "earliest"
    "key.deserializer": "org.apache.kafka.common.serialization.StringDeserializer",
    "value.deserializer": "org.apache.kafka.common.serialization.StringDeserializer"
}

In [5]:
#df = spark.read \
#    .format("kafka") \
#    .option("kafka.bootstrap.servers", "172.19.0.3:9092") \
#    .option("subscribe", "test") \
#    .load()

In [6]:
#df.show()

In [7]:
# Lee datos desde Kafka
df = spark.readStream \
    .format("kafka") \
    .options(**kafkaParams)\
    .load()

df.selectExpr("CAST(key AS STRING)", "CAST(value AS STRING)") \
  .writeStream \
  .format("kafka") \
  .trigger(processingTime="10 seconds") \
  .option('topic', "test") \
  .option('checkpointLocation', './spark_kafka_checkpoint')\
  .start() \
  .awaitTermination()


df.show()

StreamingQueryException: [STREAM_FAILED] Query [id = d1f88006-d74b-4548-b7a0-37dbf266c155, runId = 564e3d93-484d-49c8-b866-94c93ffee7e8] terminated with exception: org.apache.kafka.common.errors.TimeoutException: Call(callName=describeTopics, deadlineMs=1735090371904, tries=1, nextAllowedTryMs=1735090372006) timed out at 1735090371906 after 1 attempt(s)